# Deep Learning

Neste ficheiro, iremos explorar a possibilidade de aplicar a estratégia do 'one-hot encoding' para as dadas sequências proteicas. Esta é a representação preferida pelos modelos baseados em redes convolucionais (ou CNNs), que iremos testar.

In [19]:
import numpy as np
import pandas as pd

#### Read data and preprocess

Começamos com o mesmo preprocessamento efetuado no trabalho principal.

In [20]:
train = pd.read_csv("Files/train.csv")
updates = pd.read_csv("Files/train_updates_20220929.csv")

In [21]:
mask = updates["pH"].isna()
to_delete = updates.loc[mask,:]
to_change = updates.loc[-mask,:]

In [22]:
train.loc[to_change.index, ["pH", "tm"]] = updates.loc[to_change.index, ["pH", "tm"]]

In [23]:
train_cut = train.drop(to_delete.index)

In [24]:
idxs = train_cut[train_cut.isna().any(axis=1)].index
train_cut.drop(index=idxs, inplace=True)

In [25]:
train_cut.reset_index(drop=True, inplace=True)

In [26]:
data_train = train_cut[(train_cut["pH"] >= 1) & (train_cut["pH"] <= 14)]

In [27]:
data_train

,seq_id,protein_sequence,pH,data_source,tm
0,25,AAPDEITTAWPVNVGPLNPHLYTPNQMFAQSMVYEPLVKYQADGSV...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4
1,28,AARRFSGPRNQRQQGGGDPGLMHGKTVLITGANSGLGRATAAELLR...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4
2,29,AASSPEADFVKKTISSHKIVIFSKSYCPYCKKAKSVFRELDQVPYV...,7.0,doi.org/10.1038/s41592-020-0801-4,49.0
3,30,AATFAYSQSQKRSSSSPGGGSNHGWNNWGKAAALASTTPLVHVASV...,5.5,doi.org/10.1038/s41592-020-0801-4,55.6
4,33,AAVLVTFIGGLYFITHHKKEESETLQSQKVTGNGLPPKPEERWRYI...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4
...,...,...,...,...,...
25482,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8
25483,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2
25484,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6
25485,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7


#### Add sequence size column

Visto que esta técnica requer que todas as sequências sejam do mesmo tamanho (uma vez que gera novas matrizes de informação com cada filtro que percorre cada representação de sequência), vamos adicionar uma coluna ao dataset que indica o tamanho de cada sequência.

In [39]:
size = data_train["protein_sequence"].apply(lambda x: len(x), 0)
data_train["sequence_size"] = size.astype("float64")

#print(type(data_train["pH"][0]))
#print(type(data_train["sequence_size"][0]))

<class 'numpy.float64'>
<class 'numpy.float64'>


In [40]:
data_train = data_train.loc[:, ["seq_id", "protein_sequence", "sequence_size", "pH", "tm"]]

In [47]:
data_train.describe().loc[:,("sequence_size", "pH", "tm")]

,sequence_size,pH,tm
count,25487.000000,25487.000000,25487.000000
mean,461.254561,6.903579,51.436933
std,422.439420,0.752407,12.190382
min,5.000000,1.990000,0.000000
25%,219.000000,7.000000,43.650000
50%,359.000000,7.000000,48.700000
75%,547.000000,7.000000,54.500000
max,8798.000000,11.000000,130.000000


Observamos nesta tabela de resumo que a sequência com o maior tamanho tem 8798 resíduos, sendo um outlier, pelo que estender cada sequência a esse tamanho e aplicar a técnica de 'one-hot encoding' iria requerer imenso poder computacional. Desta forma, prosseguimos com a remoção dos outliers em relação ao tamanho das sequências.

#### Remove 'sequence_size' upper bound outliers

In [48]:
ub = np.mean(size) + np.std(size)*3
ub

1728.5479600038577

In [51]:
data_train_short = data_train[data_train["sequence_size"] < ub]


data_train_short

,seq_id,protein_sequence,sequence_size,pH,tm
0,25,AAPDEITTAWPVNVGPLNPHLYTPNQMFAQSMVYEPLVKYQADGSV...,501.0,7.0,48.4
1,28,AARRFSGPRNQRQQGGGDPGLMHGKTVLITGANSGLGRATAAELLR...,313.0,7.0,48.4
2,29,AASSPEADFVKKTISSHKIVIFSKSYCPYCKKAKSVFRELDQVPYV...,109.0,7.0,49.0
3,30,AATFAYSQSQKRSSSSPGGGSNHGWNNWGKAAALASTTPLVHVASV...,329.0,5.5,55.6
4,33,AAVLVTFIGGLYFITHHKKEESETLQSQKVTGNGLPPKPEERWRYI...,278.0,7.0,48.4
...,...,...,...,...,...
25482,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,549.0,7.0,51.8
25483,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,469.0,7.0,37.2
25484,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,128.0,7.0,64.6
25485,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,593.0,7.0,50.7


In [ ]:
#Dependent variable split

y_train = data_train_short.loc[:,"ph"]

Foram removidas 456 amostras (25487 - 25031) que tinham um tamanho de sequência superior à média + 3 vezes o desvio padrão (que consideramos como outlier)

Desta forma, ficamos com um tamanho máximo de sequência igual a:

In [50]:
max(data_train_short["sequence_size"])

1728.0

#### Extend sequences

Podemos agora estender as sequências restantes para terem todas o tamanho igual a 1728 resíduos. Vamos adicionar o símbolo "-" ao final de cada sequência de menor tamanho até atingirem o tamanho máximo.

In [53]:
max_len = len(max(data_train_short["protein_sequence"], key=len))
max_len

1728

In [54]:
for i, row in data_train_short.iterrows():
    seq = row["protein_sequence"]
    while len(seq) < max_len:
        seq += "-"
    data_train.iloc[i, 1] = seq

#### One-hot encode sequences

#### Consumo de RAM:
num_sequences = 25031
<br>max_len = 1728
<br>alphabet_len = 21 characters
<br>cell_size = 64 bits (int64)
<br>total_bits = num_sequences * max_len * alphabet_len * cell_cize = 58132795392 bits
<br><b>total_gb = 6.77 Gb</b>
<br><b>prev_total_gb = 37.67 Gb</b> (Sem remoção de outliers)

Com esta remoção de outliers, foi possível poupar **30.9 Gb de RAM**. Contudo, continua a ser um valor elevado, pelo que não é aconselhável a correr o código do seguimento, onde iremos criar um dataset com a nova representação

In [55]:
def one_hot_encoding(seq):
    alphabet = "ACDEFGHIKLMNPQRSTVWY-"
    one_hot = {}
    for i, l1 in enumerate(seq):
        for l2 in alphabet:
            one_hot[f"{l2}{i}"] = int(l1 == l2)
    return one_hot

In [1]:
# DO NOT RUN !!!
one_hot = [one_hot_encoding(seq) for seq in data_train["protein_sequence"]]

#### Create new pd.DataFrame

In [34]:
data_train_one_hot = pd.DataFrame.from_records(one_hot)

In [ ]:
data_train_one_hot

Pode-se posteriormente utilizar este novo dataset para efetuar as seguintes análises de Deep Learning. Serve como representação de uma possível metodologia que tem demonstrado bons resultados, mas por uma questão de limitações computacionais, não iremos prosseguir com a análise utilizando este dataset.

## Convolutional Neural Network

De forma semelhante ao que se fez com as redes de camadas densas, para aplicar redes convolucionais, vamos criar funções para compilar as redes de acordo com os hiperparâmetros selecionados. Desta forma, será possível efetuar uma otimização dos valores dos mesmos para encontrar o melhor conjunto de valores (que neste caso não vai ser possível averiguar).

In [2]:
from keras.layers import Conv1D, MaxPooling1D, Dense, BatchNormalization, Dropout
from keras.models import Sequential
from keras.constraints import MaxNorm
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
import tensorflow as tf
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasRegressor

import itertools

In [ ]:


def _choose_optimizer(optimizer, learning_rate, momentum):
    if optimizer == "sgd":
        return SGD(learning_rate=learning_rate, momentum=momentum)
    elif optimizer == "rmsprop":
        return RMSprop(learning_rate=learning_rate, momentum=momentum)
    elif optimizer == "adagrad":
        return Adagrad(learning_rate=learning_rate)
    elif optimizer == "adadelta":
        return Adadelta(learning_rate=learning_rate)
    elif optimizer == "adam":
        return Adam(learning_rate=learning_rate)
    elif optimizer == "adamax":
        return Adamax(learning_rate=learning_rate)
    elif optimizer == "nadam":
        return Nadam(learning_rate=learning_rate)
    else:
        raise ValueError("Unrecognized optimizer")


def create_model(input_shape, kernels, filters, kernel_size, activation, max_len, neurons, weight_constraint, dropout_rate, learning_rate, momentum, init_mode='uniform', optimizer='adam'):
    # create model
    model = Sequential()

    for ix in range(kernels):
        if ix == 0:
            model.add(Conv1D(filters,
                             kernel_size[ix],
                             activation=activation,
                             input_shape=input_shape))
        else:
            model.add(Conv1D(filters,
                             kernel_size[ix],
                             activation=activation))

        model.add(MaxPooling1D(pool_size= max_len - kernel_size[ix] + 1))

    model.add(BatchNormalization())
    #Only 1 Dense layer at the end, for simplicity purposes
    model.add(Dense(neurons,
                    activation=activation,
                    kernel_initializer=init_mode,
                    kernel_constraint=MaxNorm(weight_constraint)))

    model.add(Dropout(dropout_rate))

    model.add(Dense(1,
                    activation=activation,
                    kernel_initializer=init_mode))

    opt = _choose_optimizer(optimizer.lower(), learning_rate, momentum)

    model.compile(loss="mse", optimizer=opt, metrics=["mse"])
    return model

In [ ]:
#Parameter values to optimize:
kernels = [4,6,8]
filters = np.linspace(20, 200, 10)
kernel_size = np.linspace(5, 20, 4)
learning_rate = np.linspace(0.005, 0.5, 100)
momentum = np.linspace(0.0, 0.9, 4)
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

#(for second-to-last Dense layer)
neurons = [20,50,100]
dropout_rate = np.linspace(0.0, 0.5, 6)
weight_constraint = np.linspace(0.5, 5, 10) #Max norm value each weight parameter can be

#For computational purposes, all layers will have the same  weight_constraint and dropout_rate values

param_dist = dict(model__filters=filters,
                  model__kernel_size=kernel_size,
                  model__neurons=neurons,
                  model__dropout_rate=dropout_rate,
                  model__weight_constraint=weight_constraint,
                  model__learning_rate=learning_rate,
                  model__momentum=momentum,
                  model__optimizer=optimizer)

for ix in [4,6,8]:
    print(f"Retrieving best parameters for a {ix} layered convolutional neural network (n_iter=50, cv=5):\n|")
    model = KerasRegressor(model=create_model, epochs=100, batch_size=10, verbose=0,
                       input_shape=[data_train_one_hot.shape[1]],
                       kernels=ix,
                       max_len=max_len,
                       activation="relu")

    grid = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=50, n_jobs=-1, cv=5)
    grid.fit(data_train_one_hot, y_train)
    print(f"Best score: {grid.best_score_}")
    print(f"Best parameters: {grid.best_params_}")